# Homework 2
Sam Friedman

## Problem 1
Find d-separated nodes. Assume that we have a Bayesian network graph $\mathit{G}$. Given a set of observed nodes $\mathbf{E}$, we want to find the set of nodes $\mathit{Y}$ that contains all nodes that are d-separated from the source node $X$.
    
    

In [45]:
# Create a Graph object
class Graph(object):
    ''' Graph object that has sets of nodes and edges '''
    def __init__(self, edges=set(), nodes=set(), observations=set()):
        self.parents = {}
        self.children = {}
        self.nodes = nodes
        if len(nodes) > 0:
            self.size = len(nodes)
        else:
            self.size = 0
        self.edges = edges
        self.obs = observations

        self.build_parent_and_child_sets()
    
    def add_node(self, node):
        ''' Add node to graph '''
        if node not in self.nodes:
            self.size += 1
            self.nodes.add((node))
        else:
            pass
    
    def build_parent_and_child_sets(self):
        for edge in self.edges:
            u, v = edge
            self.add_node(u)
            self.add_node(v)
                
            if v not in self.parents:
                self.parents.update({v : set(u)})
            else:
                self.parents[v].add(u)

            if u not in self.children:
                self.children.update({u : set(v)})
            else:
                self.children[u].add(v)
                
    def get_parents(self, node):
        ''' Return parents of node '''
        if node in self.parents:
            return self.parents[node]
        else:
            return set()
    
    def get_children(self, node):
        ''' Return children of node '''
        if node in self.children:
            return self.children[node]
        else:
            return set()
                
    def find_ancestors(self, source_node):
        ''' Find all ancestors of node '''
        visited = set()  # initialize ancestor set
        queue = [source_node]              
        while queue:
            vertex = queue.pop(0)
            if vertex not in visited:
                visited.add(vertex)
                queue.extend(self.get_parents(vertex)-visited)
        return visited
        
                
    def d_separated(self, node, obs):
        ''' 
        Find all nodes that are d_separated from node given observation
        Based on Algorithm 3.1, page 75, in PGM book
        '''
        # Phase 1: insert all ancestors of Z into A
        L = obs  # set L to be the set of observations
        pass       

In [46]:
# Build Graph from homework 1, problem 3
nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
edges = set([('A','D'),('B','D'),('D','G'),('D','H'),('G','K'),('H','K'),\
             ('H','E'),('C','E'),('E','I'),('F','I'),('F','J'),('I','L'),\
             ('J','M')])
G = Graph(edges=edges)
# print('children', G.children)
# print('parents', G.parents)
# print('nodes', G.nodes)
# print('size', G.size)

start = 'L'
visited, stack = set(), [start]
while stack:
    vertex = stack.pop()
    if vertex not in visited:
        visited.add(vertex)
        stack.extend(G.get_parents(vertex) - visited)
print('end visited {}'.format(visited))
a1 = visited
a2 = G.find_ancestors('L')
a1 == a2

# # find ancestors
# print('find ancestors')
# print(G.find_ancestors('L'))
# print('--find ancestors')

# # part (f)
# Z = set(('K','E'))
# dsep_nodes = G.d_separated('A',Z)
# print('dsep nodes:', dsep_nodes)


end visited {'L', 'E', 'B', 'F', 'I', 'A', 'H', 'D', 'C'}


True

In [29]:
nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
print(nodes)

{'L', 'D', 'I', 'E', 'M', 'K', 'A', 'G', 'C', 'B', 'F', 'H', 'J'}


In [41]:
s = set()
s.add('a')
print(s)
s.add('b')
print(s)
s.add('a')
s

l = []
l.append('b')
print(l)
l2 = set(l)
print(l2)
print(isinstance(l2, list))
isinstance(l2, set)

print(l2)
l2.add({})
print(l2)

{'a'}
{'b', 'a'}
['b']
{'b'}
False
{'b'}


TypeError: unhashable type: 'dict'